In [218]:
# Boilerplate for AI Assignment — Knowledge Representation, Reasoning and Planning
# CSE 643

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import networkx as nx
from pyDatalog import pyDatalog
from collections import defaultdict, deque

## ****IMPORTANT****
## Don't import or use any other libraries other than defined above
## Otherwise your code file will be rejected in the automated testing

# ------------------ Global Variables ------------------
route_to_stops = defaultdict(list)  # Mapping of route IDs to lists of stops
trip_to_route = {}                   # Mapping of trip IDs to route IDs
stop_trip_count = defaultdict(int)    # Count of trips for each stop
fare_rules = {}                      # Mapping of route IDs to fare information
merged_fare_df = None                # To be initialized in create_kb()

# Load static data from GTFS (General Transit Feed Specification) files
df_stops = pd.read_csv('GTFS/stops.txt')
df_routes = pd.read_csv('GTFS/routes.txt')
df_stop_times = pd.read_csv('GTFS/stop_times.txt')
df_fare_attributes = pd.read_csv('GTFS/fare_attributes.txt')
df_trips = pd.read_csv('GTFS/trips.txt')
df_fare_rules = pd.read_csv('GTFS/fare_rules.txt')

# ------------------ Function Definitions ------------------

# Function to create knowledge base from the loaded data
def create_kb():
    """
    Create knowledge base by populating global variables with information from loaded datasets.
    It establishes the relationships between routes, trips, stops, and fare rules.
    
    Returns:
        None
    """
    global route_to_stops, trip_to_route, stop_trip_count, fare_rules, merged_fare_df

    # Create trip_id to route_id mapping"""
    trip_to_route = defaultdict(list)
    for _, row in df_trips.iterrows():
        trip_to_route[row['trip_id']].append(row['route_id'])
        
    # Map route_id to a list of stops in order of their sequence"""
    route_to_stops = defaultdict(list)
    sorted_stop_times = df_stop_times.sort_values(['trip_id', 'stop_sequence'])
    sorted_stop_times.head
    for trip_id, stop_grp in sorted_stop_times.groupby('trip_id'):
        if trip_id in trip_to_route:
            route_id = trip_to_route[trip_id][0]
            stops = stop_grp['stop_id'].to_list()
            route_to_stops[route_id].extend(stops)
            
    # Ensure each route only has unique stops"""
    for route_id in route_to_stops:
        # Use dict.fromkeys() to preserve order while removing duplicates
        route_to_stops[route_id] = list(dict.fromkeys(route_to_stops[route_id]))
    
    # Count trips per stop"""
    stop_trip_count = dict(df_stop_times['stop_id'].value_counts())

    # Create fare rules for routes
    fare_rules = {}
    for i in range(len(df_fare_rules['route_id'])):
        route_id = df_fare_rules['route_id'][i]
        fare_id = df_fare_rules['fare_id'][i]

        if route_id not in fare_rules:
            fare_rules[route_id] = []

        fare_rules[route_id].append(fare_id)

    # Merge fare rules and attributes into a single DataFrame
    merged_fare_df = pd.merge(
        df_fare_rules,
        df_fare_attributes,
        on='fare_id',
        how='left'
    )

# Function to find the top 5 busiest routes based on the number of trips
def get_busiest_routes():
    """
    Identify the top 5 busiest routes based on trip counts.

    Returns:
        list: A list of tuples, where each tuple contains:
              - route_id (int): The ID of the route.
              - trip_count (int): The number of trips for that route.
    """
    route_trip_counts = defaultdict(int)
    for trip_id, routes in trip_to_route.items():
        # Since we stored routes as a list in trip_to_route, take the first route
        route_id = routes[0]
        route_trip_counts[route_id] += 1
    
    res = sorted(route_trip_counts.items(), key = lambda ele: ele[1], reverse = True)
    
    # Return top 5 routes
    return res[:5]
    pass  # Implementation here

# Function to find the top 5 stops with the most frequent trips
def get_most_frequent_stops():
    """
    Identify the top 5 stops with the highest number of trips.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (int): The ID of the stop.
              - trip_count (int): The number of trips for that stop.
    """
    res = sorted(stop_trip_count.items(), key = lambda ele: ele[1], reverse = True)
    
    # Return top 5 routes
    return res[:5]
    pass  # Implementation here

# Function to find the top 5 busiest stops based on the number of routes passing through them
def get_top_5_busiest_stops():
    """
    Identify the top 5 stops with the highest number of different routes.

    Returns:
        list: A list of tuples, where each tuple contains:
              - stop_id (int): The ID of the stop.
              - route_count (int): The number of routes passing through that stop.
    """
    stop_routes = defaultdict(set)
    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            stop_routes[stop_id].add(route_id)
            
    stop_counts = []
    for stop_id, routes in stop_routes.items():
        route_count = len(routes)
        stop_counts.append((stop_id, route_count))

    res = sorted(stop_counts, key = lambda ele: ele[1], reverse = True)
    
    return res[:5]
    pass  # Implementation here

# Function to identify the top 5 pairs of stops with only one direct route between them
def get_stops_with_one_direct_route():
    """
    Identify the top 5 pairs of consecutive stops (start and end) connected by exactly one direct route. 
    The pairs are sorted by the combined frequency of trips passing through both stops.

    Returns:
        list: A list of tuples, where each tuple contains:
              - pair (tuple): A tuple with two stop IDs (stop_1, stop_2).
              - route_id (int): The ID of the route connecting the two stops.
    """
    connections = {}
    
    for route_id, stops in route_to_stops.items():
        for i in range(len(stops) - 1):
            current_stop = stops[i]
            next_stop = stops[i + 1]
            
            # same order for consistency
            stop_pair = (current_stop, next_stop) if current_stop < next_stop else (next_stop, current_stop)

            
            # Add the route to this pair's list of routes
            if stop_pair not in connections:
                connections[stop_pair] = []
            connections[stop_pair].append(route_id)
    
    result = []
    for stop_pair, routes in connections.items():
        if len(routes) == 1:
            result.append((stop_pair, routes[0]))
    
    return result
    pass  # Implementation here

# Function to get merged fare DataFrame
# No need to change this function
def get_merged_fare_df():
    """
    Retrieve the merged fare DataFrame.

    Returns:
        DataFrame: The merged fare DataFrame containing fare rules and attributes.
    """
    global merged_fare_df
    return merged_fare_df

# Visualize the stop-route graph interactively
def visualize_stop_route_graph_interactive(route_to_stops):
    """
    Visualize the stop-route graph using Plotly for interactive exploration.

    Args:
        route_to_stops (dict): A dictionary mapping route IDs to lists of stops.

    Returns:
        None
    """
    # Step 1: Create a network graph
    grph = nx.Graph()
    
    # Step 2: Add edges (connections between stops) from each route
    edge_colors = []
    edge_labels = []
    
    unique_routes = list(route_to_stops.keys())
    color_palette = plt.cm.get_cmap('hsv')(np.linspace(0, 1, len(unique_routes)))
    
    rt_color_map = {}
    for i in range (len(unique_routes)):
        route_id = unique_routes[i]
        r, g, b, _ = color_palette[i]
        
        color_string = f'rgb({int(r*255)},{int(g*255)},{int(b*255)})'
        rt_color_map[route_id] = color_string
    
    for route_id, stops_on_route in route_to_stops.items():
        route_name = df_routes[df_routes['route_id'] == route_id]['route_long_name'].iloc[0]
        for i in range(len(stops_on_route) - 1):
            current_stop = stops_on_route[i]
            next_stop = stops_on_route[i + 1]
            
            grph.add_edge(current_stop, next_stop)
            
            edge_colors.append(rt_color_map[route_id])
            edge_labels.append(f"Route: {route_name}")
        
    # Step 3: Calculate state_positions for the graph
    stop_positions = nx.shell_layout(grph)
    
    # Step 4: Create edge trace
    edge_trace = []
    for i in range (len(grph.edges())):
        edge = list(grph.edges())[i]
        color = edge_colors[i]
        label = edge_labels[i]
        
        x0, y0 = stop_positions[edge[0]]
        x1, y1 = stop_positions[edge[1]]
        
        
        edge_trace.append(
            go.Scatter(
                x=[x0, x1, None],
                y=[y0, y1, None],
                line=dict(width=2, color=color),
                hoverinfo='text',
                text=label,
                mode='lines'
            )
        )
        
    # Step 5: Create node trace
    node_x = []
    node_y = []
    node_text = []
    
    for node in grph.nodes():
        x, y = stop_positions[node]
        node_x.append(x)
        node_y.append(y)
        # Get stop name for hover text
        stop_name = df_stops[df_stops['stop_id'] == node]['stop_name'].iloc[0]
        node_text.append(f"Stop: {stop_name}<br>ID: {node}")
    
    node_trace = go.Scatter(
        x=node_x,
        y=node_y,
        mode='markers+text',
        hoverinfo='text',
        text=node_text,
        marker=dict(
            size=10,
            color='lightblue',
            line=dict(width=2, color='darkblue')
        )
    )
    
    # Step 6: Create the figure
    fig = go.Figure(data=edge_trace + [node_trace],
                   layout=go.Layout(
                       title='Transit Network Map',
                       showlegend=False,
                       hovermode='closest',
                       margin=dict(b=0, l=0, r=0, t=40),
                       xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                       yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                       plot_bgcolor='white'
                   ))
    
    # Add a legend showing route colors
    for route_id in unique_routes:
        route_name = df_routes[df_routes['route_id'] == route_id]['route_long_name'].iloc[0]
        fig.add_trace(go.Scatter(
            x=[None],
            y=[None],
            mode='lines',
            name=route_name,
            line=dict(color=rt_color_map[route_id], width=2),
            showlegend=True
        ))
    
    # Step 7: Show the plot
    fig.show()
    pass  # Implementation here

# Brute-Force Approach for finding direct routes
def direct_route_brute_force(start_stop, end_stop):
    """
    Find all valid routes between two stops using a brute-force method.

    Args:
        start_stop (int): The ID of the starting stop.
        end_stop (int): The ID of the ending stop.

    Returns:
        list: A list of route IDs (int) that connect the two stops directly.
    """
    
    dir_routes = []
    for route_id, stops in route_to_stops.items():
        if start_stop in stops and end_stop in stops:
            # Check if start_stop comes before end_stop in the sequence
            if stops.index(start_stop) < stops.index(end_stop):
                dir_routes.append(route_id)
    return dir_routes
    
    
    
    # dir_routes = []
    # for route_id, stops in route_to_stops.items():
    #     start_indexes = [i for i, stop in enumerate(stops) if stop == start_stop]
    #     end_indexes = [i for i, stop in enumerate(stops) if stop == end_stop]
    #     for start_idx in start_indexes:
    #         for end_idx in end_indexes:
    #             if end_idx > start_idx:
    #                 dir_routes.append(route_id)
    #                 break
    #         if route_id in dir_routes:
    #             break
            
    # return sorted(dir_routes)
    
    pass  # Implementation here

# Initialize Datalog predicates for reasoning
pyDatalog.create_terms('RouteHasStop, DirectRoute, OptimalRoute, X, Y, Z, R, R1, R2')  
def initialize_datalog():
    """
    Initialize Datalog terms and predicates for reasoning about routes and stops.

    Returns:
        None
    """
    pyDatalog.clear()  # Clear previous terms
    print("Terms initialized: DirectRoute, RouteHasStop, OptimalRoute")  # Confirmation print

    # Define Datalog predicates

    create_kb()  # Populate the knowledge base
    add_route_data(route_to_stops)  # Add route data to Datalog
    
    
# Adding route data to Datalog
def add_route_data(route_to_stops):
    """
    Add the route data to Datalog for reasoning.

    Args:
        route_to_stops (dict): A dictionary mapping route IDs to lists of stops.

    Returns:
        None
    """
    for route_id, stops in route_to_stops.items():
        for stop_id in stops:
            +RouteHasStop(route_id, stop_id)

# Function to query direct routes between two stops
def query_direct_routes(start, end):
    """
    Query for direct routes between two stops.

    Args:
        start (int): The ID of the starting stop.
        end (int): The ID of the ending stop.

    Returns:
        list: A sorted list of route IDs (str) connecting the two stops.
    """
    
    # Query for routes that contain both start and end stops
    query_result = pyDatalog.ask(f'RouteHasStop(R, {start}) & RouteHasStop(R, {end})')
    
    # Process the results, assuming each answer contains a single route_id
    if query_result:
        return sorted(set(route_id[0] for route_id in query_result.answers))
    return []
    
    pass  # Implementation here


In [219]:
from collections import defaultdict

route_to_stops = defaultdict(list)  # Maps route_id to an ordered list of stop_ids
trip_to_route = {}  # Maps trip_id to route_id
stop_trip_count = defaultdict(int)  # Maps stop_id to count of trips stopping there
fare_rules = {}  # Maps route_id to fare information

In [220]:
create_kb()  # Ensure the data is loaded before testing
merged_fare_df = get_merged_fare_df()  # Use the function to retrieve the DataFrame
initialize_datalog()

Terms initialized: DirectRoute, RouteHasStop, OptimalRoute


In [221]:
route_to_stops[294]

[1323,
 2713,
 1324,
 1325,
 1326,
 1327,
 1328,
 2714,
 1329,
 1330,
 22511,
 2600,
 2601,
 2602,
 2603,
 2604,
 1159,
 1160,
 1161,
 1162,
 22530,
 973,
 974,
 975,
 22532,
 22533,
 22534,
 22535,
 4051,
 981,
 982,
 983,
 984,
 985,
 986,
 987,
 988,
 989,
 990,
 991,
 992,
 993,
 994,
 995,
 996,
 997,
 2605,
 2606,
 2607,
 2608,
 2609,
 2009,
 4511,
 1438,
 1440,
 1441,
 951,
 952,
 953,
 954,
 955,
 293,
 3436,
 3437,
 3438,
 3439,
 3440,
 3441,
 42,
 43,
 44,
 936,
 46,
 47,
 48,
 108,
 109,
 110,
 111]

In [222]:
# Sample public test inputs with expected outputs explicitly defined
test_inputs = {
    "direct_route": [
        ((2573, 1177), [10001, 1117, 1407]),  # Input -> Expected output
        ((2001, 2005), [10001, 1151])
    ],

    "forward_chaining": [
        ((22540, 2573, 4686, 1), [(10153, 4686, 1407)]),
        ((951, 340, 300, 1), [(294, 300, 712), (10453, 300, 712), (1211, 300, 712), (1158, 300, 712), 
                              (37, 300, 712), (1571, 300, 712), (49, 300, 712), (387, 300, 712), 
                              (1206, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)])
    ],
    "backward_chaining": [
        ((2573, 22540, 4686, 1), [(1407, 4686, 10153)]),
        ((340, 951, 300, 1), [(712, 300, 121), (712, 300, 1211), (712, 300, 37), (712, 300, 387),
                              (712, 300, 49), (712, 300, 10453), (712, 300, 1038), (712, 300, 10433),
                              (712, 300, 1571)])
    ],
    "pddl_planning": [
        ((22540, 2573, 4686, 1), [(10153, 4686, 1407)]),
        ((951, 340, 300, 1), [(294, 300, 712), (10453, 300, 712), (1211, 300, 712), (1158, 300, 712), 
                              (37, 300, 712), (1571, 300, 712), (49, 300, 712), (387, 300, 712), 
                              (1206, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)])
    ],
    "bfs_route": [
        ((22540, 2573, 10, 3), [(10153, 4686), (1407, 2573)]),
        ((4012, 4013, 10, 3), [(10004, 4013)])
    ],

    ### NOTE: The below values are just dummy values, the actual values are might differ! 
    "busiest_routes": [
        [(123, 456), (789, 234), (567, 235), (3456, 897), (345, 345)]
    ],
    "most_frequent_stops": [
        [(456, 456), (234, 765), (234, 765), (234, 657765), (3252, 35634)]
    ],
    "busiest_stops": [
        [(432243, 14543), (454235, 2452), (2452, 2454), (78568, 24352), (42352, 24532)]
    ],
    "stops_with_one_direct_route": [
        [((24527, 676), 542), ((243535, 8768), 2456), ((43262, 564), 65437),
         ((256, 56), 245), ((266, 256), 78)]
    ]
}

def check_output(expected, actual):
    """Function to compare expected and actual outputs."""
    return set(expected) == set(actual)

def test_direct_route_brute_force():
    for (start_stop, end_stop), expected_output in test_inputs["direct_route"]:
        actual_output = direct_route_brute_force(start_stop, end_stop)
        print(f"Test direct_route_brute_force ({start_stop}, {end_stop}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_query_direct_routes():
    for (start_stop, end_stop), expected_output in test_inputs["direct_route"]:
        actual_output = query_direct_routes(start_stop, end_stop)
        print(f"Test query_direct_routes ({start_stop}, {end_stop}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_forward_chaining():
    for (start_stop, end_stop, via_stop, max_transfers), expected_output in test_inputs["forward_chaining"]:
        actual_output = forward_chaining(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test forward_chaining ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_backward_chaining():
    for (end_stop, start_stop, via_stop, max_transfers), expected_output in test_inputs["backward_chaining"]:
        actual_output = backward_chaining(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test backward_chaining ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_pddl_planning():
    for (start_stop, end_stop, via_stop, max_transfers), expected_output in test_inputs["pddl_planning"]:
        actual_output = pddl_planning(start_stop, end_stop, via_stop, max_transfers)
        print(f"Test pddl_planning ({start_stop}, {end_stop}, {via_stop}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_bfs_route_planner():
    for (start_stop, end_stop, initial_fare, max_transfers), expected_output in test_inputs["bfs_route"]:
        pruned_df = prune_data(merged_fare_df, initial_fare)
        route_summary = compute_route_summary(pruned_df)
        actual_output = bfs_route_planner_optimized(start_stop, end_stop, initial_fare, route_summary, max_transfers)
        print(f"Test bfs_route_planner_optimized ({start_stop}, {end_stop}, {initial_fare}, {max_transfers}): ", 
              "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

# New test functions for the additional queries

def test_get_busiest_routes():
    expected_output = test_inputs["busiest_routes"][0]
    actual_output = get_busiest_routes()
    print(f"Test get_busiest_routes: ", 
          "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_get_most_frequent_stops():
    expected_output = test_inputs["most_frequent_stops"][0]
    actual_output = get_most_frequent_stops()
    print(f"Test get_most_frequent_stops: ", 
          "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_get_top_5_busiest_stops():
    expected_output = test_inputs["busiest_stops"][0]
    actual_output = get_top_5_busiest_stops()
    print(f"Test get_top_5_busiest_stops: ", 
          "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

def test_get_stops_with_one_direct_route():
    expected_output = test_inputs["stops_with_one_direct_route"][0]
    actual_output = get_stops_with_one_direct_route()
    print(f"Test get_stops_with_one_direct_route: ", 
          "Pass" if check_output(expected_output, actual_output) else f"Fail (Expected: {expected_output}, Got: {actual_output})")

# if __name__ == "__main__":
#     create_kb()  # Ensure the data is loaded before testing
#     merged_fare_df = get_merged_fare_df()  # Use the function to retrieve the DataFrame
#     initialize_datalog()
    
#     # Run all tests
#     test_direct_route_brute_force()
#     test_query_direct_routes()
#     test_forward_chaining()
#     test_backward_chaining()
#     test_pddl_planning()
#     test_bfs_route_planner()
    
#     # Run additional tests for the new queries
#     test_get_busiest_routes()
#     test_get_most_frequent_stops()
#     test_get_top_5_busiest_stops()
#     test_get_stops_with_one_direct_route()

In [223]:
test_direct_route_brute_force()
test_query_direct_routes()

Test direct_route_brute_force (2573, 1177):  Pass
Test direct_route_brute_force (2001, 2005):  Pass
Test query_direct_routes (2573, 1177):  Pass
Test query_direct_routes (2001, 2005):  Pass


In [224]:
def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Perform forward chaining to find optimal routes considering transfers.
    
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID that must be included in the path.
        max_transfers (int): Maximum number of transfers allowed.
    
    Returns:
        list: List of paths that satisfy the constraints.
    """
    import time
    import psutil
    start_time = time.time()
    
    paths = []
    visited_routes = set()
    
    # Step 1: Find all routes that contain the start stop
    start_routes = set()
    for route_id, stops in route_to_stops.items():
        if start_stop_id in stops:
            start_routes.add(route_id)
    
    # Step 2: For each starting route, try to find paths
    for start_route in start_routes:
        if start_route in visited_routes:
            continue
            
        visited_routes.add(start_route)
        stops = route_to_stops[start_route]
        
        try:
            start_idx = stops.index(start_stop_id)
            
            # Check if the via stop is in this route
            if stop_id_to_include in stops:
                via_idx = stops.index(stop_id_to_include)
                
                # Check if end stop is also in this route
                if end_stop_id in stops:
                    end_idx = stops.index(end_stop_id)
                    
                    # Verify the sequence is valid (start -> via -> end or start -> end -> via)
                    if ((start_idx < via_idx < end_idx) or 
                        (start_idx > via_idx > end_idx)):
                        path = [(start_route, stop) for stop in stops[min(start_idx, end_idx):max(start_idx, end_idx) + 1]]
                        paths.append(path)
                else:
                    # Need to find connecting routes
                    for route2, stops2 in route_to_stops.items():
                        if route2 != start_route and stop_id_to_include in stops2 and end_stop_id in stops2:
                            # Valid transfer found
                            if len(paths) == 0 or len(paths[-1]) <= max_transfers + 1:
                                path1 = [(start_route, stop) for stop in stops[start_idx:via_idx + 1]]
                                end_idx2 = stops2.index(end_stop_id)
                                via_idx2 = stops2.index(stop_id_to_include)
                                path2 = [(route2, stop) for stop in stops2[via_idx2:end_idx2 + 1]]
                                paths.append(path1 + path2[1:])  # Avoid duplicating via stop
        
        except ValueError:
            continue
    
    # Calculate execution metrics
    end_time = time.time()
    execution_time = end_time - start_time
    memory_usage = psutil.Process().memory_info().rss / 1024 / 1024  # Convert to MB
    
    print(f"Forward Chaining Metrics:")
    print(f"Execution Time: {execution_time:.4f} seconds")
    print(f"Memory Usage: {memory_usage:.2f} MB")
    print(f"Number of paths found: {len(paths)}")
    
    return paths

def backward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Perform backward chaining to find optimal routes considering transfers.
    
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID that must be included in the path.
        max_transfers (int): Maximum number of transfers allowed.
    
    Returns:
        list: List of paths that satisfy the constraints.
    """
    import time
    import psutil
    start_time = time.time()
    
    paths = []
    visited_routes = set()
    
    # Step 1: Find all routes that contain the end stop
    end_routes = set()
    for route_id, stops in route_to_stops.items():
        if end_stop_id in stops:
            end_routes.add(route_id)
    
    # Step 2: Work backwards from end stop
    for end_route in end_routes:
        if end_route in visited_routes:
            continue
            
        visited_routes.add(end_route)
        stops = route_to_stops[end_route]
        
        try:
            end_idx = stops.index(end_stop_id)
            
            # Check if via stop is in this route
            if stop_id_to_include in stops:
                via_idx = stops.index(stop_id_to_include)
                
                # Check if start stop is also in this route
                if start_stop_id in stops:
                    start_idx = stops.index(start_stop_id)
                    
                    # Verify sequence is valid
                    if ((start_idx < via_idx < end_idx) or 
                        (start_idx > via_idx > end_idx)):
                        path = [(end_route, stop) for stop in stops[min(start_idx, end_idx):max(start_idx, end_idx) + 1]]
                        paths.append(path)
                else:
                    # Need to find connecting routes from start to via
                    for route1, stops1 in route_to_stops.items():
                        if route1 != end_route and start_stop_id in stops1 and stop_id_to_include in stops1:
                            # Valid transfer found
                            if len(paths) == 0 or len(paths[-1]) <= max_transfers + 1:
                                start_idx1 = stops1.index(start_stop_id)
                                via_idx1 = stops1.index(stop_id_to_include)
                                path1 = [(route1, stop) for stop in stops1[start_idx1:via_idx1 + 1]]
                                path2 = [(end_route, stop) for stop in stops[via_idx:end_idx + 1]]
                                paths.append(path1 + path2)
        
        except ValueError:
            continue
    
    # Calculate execution metrics
    end_time = time.time()
    execution_time = end_time - start_time
    memory_usage = psutil.Process().memory_info().rss / 1024 / 1024  # Convert to MB
    
    print(f"Backward Chaining Metrics:")
    print(f"Execution Time: {execution_time:.4f} seconds")
    print(f"Memory Usage: {memory_usage:.2f} MB")
    print(f"Number of paths found: {len(paths)}")
    
    return paths


In [225]:
# def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
#     import time
#     import psutil
#     start_time = time.time()
#     paths = []

#     # Step 1: Find all routes that contain the start stop and via stop
#     start_via_routes = []
#     for route_id, stops in route_to_stops.items():
#         if start_stop_id in stops and stop_id_to_include in stops:
#             start_via_routes.append((route_id, stops.index(start_stop_id), stops.index(stop_id_to_include)))

#     # Step 2: Find all routes that contain the via stop and end stop
#     via_end_routes = []
#     for route_id, stops in route_to_stops.items():
#         if stop_id_to_include in stops and end_stop_id in stops:
#             via_end_routes.append((route_id, stops.index(stop_id_to_include), stops.index(end_stop_id)))

#     # Step 3: Combine the routes
#     for start_route, start_idx, via_idx in start_via_routes:
#         for end_route, via_idx2, end_idx in via_end_routes:
#             if start_route != end_route:  # Ensure one transfer
#                 path1 = [(start_route, stop) for stop in route_to_stops[start_route][start_idx:via_idx + 1]]
#                 path2 = [(end_route, stop) for stop in route_to_stops[end_route][via_idx2:end_idx + 1]]
#                 paths.append(path1 + path2[1:])  # Avoid duplicating via stop

#     # Calculate execution metrics
#     end_time = time.time()
#     execution_time = end_time - start_time
#     memory_usage = psutil.Process().memory_info().rss / 1024 / 1024  # Convert to MB
#     print(f"Forward Chaining Metrics:")
#     print(f"Execution Time: {execution_time:.4f} seconds")
#     print(f"Memory Usage: {memory_usage:.2f} MB")
#     print(f"Number of paths found: {len(paths)}")

#     for i, path in enumerate(paths):
#         print(f"Path {i+1}:")
#         for route_id, stop_id in path:
#             print(f"Route ID: {route_id}, Stop ID: {stop_id}")
#         print()  # Empty line for readability

#     return paths

In [226]:
# def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
#     """Perform forward chaining to find optimal routes considering transfers.
    
#     Args:
#         start_stop_id (int): The starting stop ID.
#         end_stop_id (int): The ending stop ID.
#         stop_id_to_include (int): The stop ID where a transfer occurs.
#         max_transfers (int): The maximum number of transfers allowed.
        
#     Returns:
#         list: A list of tuples (route_id, via_stop, end_stop) representing valid paths
#     """
#     result_paths = set()
    
#     # Find routes containing the required stops
#     for route_id, stops in route_to_stops.items():
#         try:
#             # Check if all required stops are in this route
#             if stop_id_to_include in stops and (start_stop_id in stops or end_stop_id in stops):
#                 via_idx = stops.index(stop_id_to_include)
                
#                 # Case 1: Direct route containing all stops
#                 if start_stop_id in stops and end_stop_id in stops:
#                     start_idx = stops.index(start_stop_id)
#                     end_idx = stops.index(end_stop_id)
                    
#                     # Check if the route is valid (stops are in correct order)
#                     if min(start_idx, end_idx) <= via_idx <= max(start_idx, end_idx):
#                         result_paths.add((route_id, stop_id_to_include, end_stop_id))
                
#                 # Case 2: Route contains via stop and either start or end
#                 elif max_transfers >= 1:
#                     # Find connecting routes
#                     for other_route, other_stops in route_to_stops.items():
#                         if other_route != route_id:
#                             if start_stop_id in stops and end_stop_id in other_stops:
#                                 if stop_id_to_include in other_stops:
#                                     result_paths.add((other_route, stop_id_to_include, end_stop_id))
#                             elif start_stop_id in other_stops and end_stop_id in stops:
#                                 if stop_id_to_include in other_stops:
#                                     result_paths.add((route_id, stop_id_to_include, end_stop_id))
                                    
#         except ValueError:
#             continue
    
#     # Convert set to list for return
#     return list(result_paths)

In [227]:
def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """Perform forward chaining to find optimal routes considering transfers.
    
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.
        
    Returns:
        list: A list of tuples (route_id, via_stop, end_stop) representing valid paths
    """
    result_paths = set()
    
    # Find routes containing the required stops
    for route_id, stops in route_to_stops.items():
        try:
            # Check if all required stops are in this route
            if stop_id_to_include in stops and (start_stop_id in stops or end_stop_id in stops):
                via_idx = stops.index(stop_id_to_include)
                
                # Case 1: Direct route containing all stops
                if start_stop_id in stops and end_stop_id in stops:
                    start_idx = stops.index(start_stop_id)
                    end_idx = stops.index(end_stop_id)
                    
                    # Check if the route is valid (stops are in correct order)
                    if min(start_idx, end_idx) <= via_idx <= max(start_idx, end_idx):
                        result_paths.add((route_id, stop_id_to_include, end_stop_id))
                        print("yes")
                
                # Case 2: Route contains via stop and either start or end
                elif max_transfers >= 1:
                    # Find connecting routes
                    for other_route, other_stops in route_to_stops.items():
                        if other_route != route_id:
                            if start_stop_id in stops and end_stop_id in other_stops:
                                if stop_id_to_include in other_stops:
                                    print((route_id, stop_id_to_include, other_route))
                                    result_paths.add((route_id, stop_id_to_include, other_route))
                                    print("yes2")
                            elif start_stop_id in other_stops and end_stop_id in stops:
                                if stop_id_to_include in other_stops:
                                    result_paths.add((route_id, stop_id_to_include, end_stop_id))
                                    print((route_id, stop_id_to_include, route_id))
                                    print("yes3")
                                    
        except ValueError:
            continue
    
    # Convert set to list for return
    return sorted(list(result_paths))

In [228]:
def forward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """Perform forward chaining to find optimal routes considering transfers.
    
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.
        
    Returns:
        list: A list of tuples (route_id, via_stop, end_stop) representing valid paths
    """
    result_paths = set()
    
    # Find routes containing the required stops
    for route_id, stops in route_to_stops.items():
        try:
            # Check if all required stops are in this route
            if stop_id_to_include in stops and (start_stop_id in stops or end_stop_id in stops):
                via_idx = stops.index(stop_id_to_include)
                # print(route_id)
                # Case 1: Direct route containing all stops
                if start_stop_id in stops and end_stop_id in stops:
                    start_idx = stops.index(start_stop_id)
                    end_idx = stops.index(end_stop_id)
                    
                    # Check if the route is valid (stops are in correct order)
                    if min(start_idx, end_idx) <= via_idx <= max(start_idx, end_idx):
                        result_paths.add((route_id, stop_id_to_include, end_stop_id))
                        # print("yes")
                        
                
                # Case 2: Route contains via stop and either start or end
                elif max_transfers >= 1:
                    # Find connecting routes
                    for other_route, other_stops in route_to_stops.items():
                        # print(other_route)
                        if other_route != route_id:
                            # print(other_route)
                            if start_stop_id in stops and end_stop_id in other_stops:
                                # print(other_route)
                                if stop_id_to_include in other_stops:
                                    
                                    result_paths.add((route_id, stop_id_to_include, other_route))
                                    # print((route_id, stop_id_to_include, other_route))
                                    # print("yes2")
                                    # print(other_route)
                                    # print(end_stop_id)
                            elif start_stop_id in other_stops and end_stop_id in stops:
                                # print((other_route, stop_id_to_include, route_id,start_stop_id, end_stop_id))
                                # print(other_stops)
                                if stop_id_to_include in other_stops:
                                    # print(other_route)
                                    # print((other_route, stop_id_to_include, route_id,start_stop_id, end_stop_id))
                                    result_paths.add((other_route, stop_id_to_include, route_id))
                                    # print((other_route, stop_id_to_include, route_id))
                                    # print("yes3")
                                    # print(other_route)
        except ValueError:
            continue
    
    # Convert set to list for return
    return sorted(list(result_paths))

In [197]:
test_forward_chaining()

Test forward_chaining (22540, 2573, 4686, 1):  Pass
Test forward_chaining (951, 340, 300, 1):  Fail (Expected: [(294, 300, 712), (10453, 300, 712), (1211, 300, 712), (1158, 300, 712), (37, 300, 712), (1571, 300, 712), (49, 300, 712), (387, 300, 712), (1206, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)], Got: [(37, 300, 712), (49, 300, 712), (121, 300, 712), (387, 300, 712), (1038, 300, 712), (1211, 300, 712), (1571, 300, 712), (10433, 300, 712), (10453, 300, 712)])


In [ ]:
# def backward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
#     """Perform backward chaining to find optimal routes considering transfers.
    
#     Args:
#         start_stop_id (int): The starting stop ID.
#         end_stop_id (int): The ending stop ID.
#         stop_id_to_include (int): The stop ID where a transfer occurs.
#         max_transfers (int): The maximum number of transfers allowed.
    
#     Returns:
#         list: A list of tuples (route_id, via_stop, end_stop) representing valid paths
#     """
#     result_paths = []
#     seen_paths = set()  # To avoid duplicates while maintaining order
    
#     # Start from the end stop and work backwards
#     for route_id, stops in route_to_stops.items():
#         try:
#             # Check if the route contains the end stop and via stop
#             if end_stop_id in stops and stop_id_to_include in stops:
#                 end_idx = stops.index(end_stop_id)
#                 via_idx = stops.index(stop_id_to_include)
                
#                 # Case 1: Direct route containing all stops
#                 if start_stop_id in stops:
#                     start_idx = stops.index(start_stop_id)
#                     # Check if the route is valid (stops are in correct order)
#                     if min(via_idx, end_idx) <= start_idx <= max(via_idx, end_idx):
#                         path_tuple = (route_id, stop_id_to_include, end_stop_id)
#                         path_key = str(path_tuple)  # Convert to string for hashing
#                         print("yes1")
#                         print(path_tuple)
#                         if path_key not in seen_paths:
#                             seen_paths.add(path_key)
#                             result_paths.append(path_tuple)
                
#                 # Case 2: Route contains end stop and via stop, need to find connecting route
#                 elif max_transfers >= 1:
#                     # Look for routes that can connect to our current route at the via stop
#                     for connecting_route, connecting_stops in route_to_stops.items():
#                         if connecting_route != route_id:
#                             # Check if connecting route has start stop and via stop
#                             if start_stop_id in connecting_stops and stop_id_to_include in connecting_stops:
#                                 conn_start_idx = connecting_stops.index(start_stop_id)
#                                 conn_via_idx = connecting_stops.index(stop_id_to_include)
                                
#                                 # Verify the order in connecting route
#                                 if min(conn_start_idx, conn_via_idx) <= max(conn_start_idx, conn_via_idx):
#                                     path_tuple = (connecting_route, stop_id_to_include, route_id)
#                                     print(path_tuple)
#                                     print("yes2")
#                                     path_key = str(path_tuple)
#                                     if path_key not in seen_paths:
#                                         seen_paths.add(path_key)
#                                         result_paths.append(path_tuple)
                                        
            
#             # Additional case: Route contains start stop and via stop
#             elif start_stop_id in stops and stop_id_to_include in stops and max_transfers >= 1:
#                 start_idx = stops.index(start_stop_id)
#                 via_idx = stops.index(stop_id_to_include)
                
#                 # Look for routes that can connect from via stop to end stop
#                 for next_route, next_stops in route_to_stops.items():
#                     if next_route != route_id:
#                         if end_stop_id in next_stops and stop_id_to_include in next_stops:
#                             next_end_idx = next_stops.index(end_stop_id)
#                             next_via_idx = next_stops.index(stop_id_to_include)
                            
#                             # Verify the order in next route
#                             if min(next_via_idx, next_end_idx) <= max(next_via_idx, next_end_idx):
#                                 path_tuple = (route_id, stop_id_to_include, next_route)
#                                 print("yes3")
#                                 print(path_tuple)
#                                 path_key = str(path_tuple)
#                                 if path_key not in seen_paths:
#                                     seen_paths.add(path_key)
#                                     result_paths.append(path_tuple)
                                
#         except ValueError:
#             continue
    
#     # Sort the results for consistent output
#     return sorted(result_paths)


In [246]:
def backward_chaining(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """Perform backward chaining to find optimal routes considering transfers.
    
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID where a transfer occurs.
        max_transfers (int): The maximum number of transfers allowed.
    
    Returns:
        list: A list of tuples (route_id, via_stop, end_stop) representing valid paths
    """
    result_paths = []
    seen_paths = set()  # To avoid duplicates while maintaining order
    
    # Start from the end stop and work backwards
    for route_id, stops in route_to_stops.items():
        try:
            # Check if the route contains the end stop and via stop
            if end_stop_id in stops and stop_id_to_include in stops:
                end_idx = stops.index(end_stop_id)
                via_idx = stops.index(stop_id_to_include)
                
                # Case 1: Direct route containing all stops
                if start_stop_id in stops:
                    start_idx = stops.index(start_stop_id)
                    # Check if the route is valid (stops are in correct order)
                    if min(via_idx, end_idx) <= start_idx <= max(via_idx, end_idx):
                        path_tuple = (end_stop_id, stop_id_to_include, route_id)
                        path_key = str(path_tuple)  # Convert to string for hashing
                        print("yes1")
                        print(path_tuple)
                        if path_key not in seen_paths:
                            seen_paths.add(path_key)
                            result_paths.append(path_tuple)
                
                # Case 2: Route contains end stop and via stop, need to find connecting route
                elif max_transfers >= 1:
                    # Look for routes that can connect to our current route at the via stop
                    for connecting_route, connecting_stops in route_to_stops.items():
                        if connecting_route != route_id:
                            # Check if connecting route has start stop and via stop
                            if start_stop_id in connecting_stops and stop_id_to_include in connecting_stops:
                                conn_start_idx = connecting_stops.index(start_stop_id)
                                conn_via_idx = connecting_stops.index(stop_id_to_include)
                                
                                # Verify the order in connecting route
                                if min(conn_start_idx, conn_via_idx) <= max(conn_start_idx, conn_via_idx):
                                    path_tuple = (route_id, stop_id_to_include, connecting_route)
                                    print("yes2")
                                    print(path_tuple)
                                    path_key = str(path_tuple)
                                    if path_key not in seen_paths:
                                        seen_paths.add(path_key)
                                        result_paths.append(path_tuple)
                                        
            
            # Additional case: Route contains start stop and via stop
            elif start_stop_id in stops and stop_id_to_include in stops and max_transfers >= 1:
                start_idx = stops.index(start_stop_id)
                via_idx = stops.index(stop_id_to_include)
                
                # Look for routes that can connect from via stop to end stop
                for next_route, next_stops in route_to_stops.items():
                    if next_route != route_id:
                        if end_stop_id in next_stops and stop_id_to_include in next_stops:
                            next_end_idx = next_stops.index(end_stop_id)
                            next_via_idx = next_stops.index(stop_id_to_include)
                            
                            # Verify the order in next route
                            if min(next_via_idx, next_end_idx) <= max(next_via_idx, next_end_idx):
                                path_tuple = (next_route, stop_id_to_include, route_id)
                                print("yes3")
                                print(path_tuple)
                                path_key = str(path_tuple)
                                if path_key not in seen_paths:
                                    seen_paths.add(path_key)
                                    result_paths.append(path_tuple)
                                
        except ValueError:
            continue
    
    # Sort the results for consistent output
    return sorted(result_paths)


In [247]:
test_backward_chaining()

yes3
(1407, 4686, 10153)
yes2
(1407, 4686, 10153)
Test backward_chaining (22540, 2573, 4686, 1):  Pass
yes3
(712, 300, 1038)
yes3
(712, 300, 10433)
yes3
(712, 300, 10453)
yes3
(712, 300, 1211)
yes3
(712, 300, 121)
yes3
(712, 300, 1571)
yes3
(712, 300, 37)
yes3
(712, 300, 387)
yes3
(712, 300, 49)
yes2
(712, 300, 1038)
yes2
(712, 300, 10433)
yes2
(712, 300, 10453)
yes2
(712, 300, 1211)
yes2
(712, 300, 121)
yes2
(712, 300, 1571)
yes2
(712, 300, 37)
yes2
(712, 300, 387)
yes2
(712, 300, 49)
Test backward_chaining (951, 340, 300, 1):  Pass


In [256]:
# import time
# import psutil
# import sys
# from datetime import datetime

# # PDDL-style planning implementation
# def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
#     """
#     Implement PDDL-style planning to find routes with optional transfers.
    
#     Uses forward chaining with PDDL-style actions and state tracking.
#     Measures execution time and memory usage for analysis.
#     """
#     start_time = time.time()
#     process = psutil.Process()
#     initial_memory = process.memory_info().rss / (1024 * 1024)  # Convert to MB
    
#     # print("\nPDDL Planning Analysis:")
#     # print(f"Initial State: Stop {start_stop_id}")
#     # print(f"Goal State: Stop {end_stop_id}")
#     # print(f"Required Transfer Stop: {stop_id_to_include}")
#     # print(f"Max Transfers Allowed: {max_transfers}\n")
    
#     class State:
#         def __init__(self, current_stop, visited_stops, current_route, transfers_used, path):
#             self.current_stop = current_stop
#             self.visited_stops = visited_stops
#             self.current_route = current_route
#             self.transfers_used = transfers_used
#             self.path = path
            
#     # Initialize actions using PyDatalog
#     pyDatalog.create_terms('Board, Transfer, ValidRoute, ValidTransfer, X, Y, Z, R, R1, R2')
    
#     # Define PDDL-style actions
#     valid_routes = []
#     visited_states = set()
#     initial_state = State(start_stop_id, {start_stop_id}, None, 0, [])
#     states_to_explore = [initial_state]
    
#     # print("Starting PDDL Planning Process...")
    
#     while states_to_explore:
#         current_state = states_to_explore.pop(0)
#         state_key = (current_state.current_stop, current_state.current_route, 
#                     current_state.transfers_used)
        
#         if state_key in visited_states:
#             continue
            
#         visited_states.add(state_key)
        
#         # print(f"\nExploring State:")
#         # print(f"Current Stop: {current_state.current_stop}")
#         # print(f"Current Route: {current_state.current_route}")
#         # print(f"Transfers Used: {current_state.transfers_used}")
        
#         # Check if goal reached
#         if (current_state.current_stop == end_stop_id and 
#             stop_id_to_include in current_state.visited_stops):
#             route_path = current_state.path
#             if route_path and len(route_path) >= 2:  # Must have at least two stops
#                 valid_routes.append(route_path)
#                 # print(f"Found Valid Route: {route_path}")
#             continue
            
#         # Action 1: Board a route
#         for route_id, stops in route_to_stops.items():
#             if current_state.current_stop in stops:
#                 next_stop_idx = stops.index(current_state.current_stop)
                
#                 # Check forward direction
#                 for next_idx in range(next_stop_idx + 1, len(stops)):
#                     next_stop = stops[next_idx]
#                     if next_stop not in current_state.visited_stops:
#                         new_path = current_state.path + [(route_id, next_stop)]
#                         new_state = State(
#                             next_stop,
#                             current_state.visited_stops | {next_stop},
#                             route_id,
#                             current_state.transfers_used,
#                             new_path
#                         )
#                         states_to_explore.append(new_state)
#                         # print(f"Action: Board route {route_id} to stop {next_stop}")
                        
#         # Action 2: Transfer between routes
#         if current_state.transfers_used < max_transfers:
#             for route_id, stops in route_to_stops.items():
#                 if (route_id != current_state.current_route and 
#                     current_state.current_stop in stops):
#                     new_state = State(
#                         current_state.current_stop,
#                         current_state.visited_stops,
#                         route_id,
#                         current_state.transfers_used + 1,
#                         current_state.path
#                     )
#                     states_to_explore.append(new_state)
#                     # print(f"Action: Transfer to route {route_id}")
                    
#     # Convert paths to required format
#     formatted_routes = []
#     for path in valid_routes:
#         for i in range(len(path) - 1):
#             if path[i][1] == stop_id_to_include:
#                 formatted_routes.append((path[i][0], path[i][1], path[i+1][0]))
                
#     # Measure performance metrics
#     end_time = time.time()
#     final_memory = process.memory_info().rss / (1024 * 1024)
#     execution_time = end_time - start_time
#     memory_usage = final_memory - initial_memory
    
#     # print("\nPerformance Metrics:")
#     # print(f"Execution Time: {execution_time:.2f} seconds")
#     # print(f"Memory Usage: {memory_usage:.2f} MB")
#     # print(f"States Explored: {len(visited_states)}")
#     # print(f"Valid Routes Found: {len(formatted_routes)}")
    
#     return sorted(list(set(formatted_routes)))

# def prune_data(merged_fare_df, initial_fare):
#     """
#     Filter fare data based on an initial fare limit.
#     """
#     # Filter routes where price is less than or equal to initial fare
#     pruned_df = merged_fare_df[merged_fare_df['price'] <= initial_fare].copy()
    
#     print(f"Pruned {len(merged_fare_df) - len(pruned_df)} routes exceeding fare limit")
#     return pruned_df

# def compute_route_summary(pruned_df):
#     """
#     Generate a summary of routes based on fare information.
#     """
#     route_summary = {}
    
#     # Group by route_id and compute minimum price
#     for route_id in pruned_df['route_id'].unique():
#         route_fares = pruned_df[pruned_df['route_id'] == route_id]
#         min_price = route_fares['price'].min()
        
#         # Get all stops for this route
#         route_stops = set(route_to_stops.get(route_id, []))
        
#         route_summary[route_id] = {
#             'min_price': min_price,
#             'stops': route_stops
#         }
        
#     return route_summary

# def bfs_route_planner_optimized(start_stop_id, end_stop_id, initial_fare, route_summary, max_transfers=3):
#     """
#     Optimized BFS route planner considering fare constraints.
#     """
#     queue = deque([(start_stop_id, [], 0, 0)])  # (stop, path, total_fare, transfers)
#     visited = set()  # Track visited (stop, route) combinations
#     valid_paths = []
    
#     while queue:
#         current_stop, path, total_fare, transfers = queue.popleft()
        
#         # Check if we've reached the destination
#         if current_stop == end_stop_id:
#             valid_paths.append((path, total_fare))
#             continue
            
#         # Get all possible routes from current stop
#         for route_id, info in route_summary.items():
#             if current_stop in info['stops']:
#                 # Skip if we've exceeded transfers
#                 if path and path[-1][0] != route_id and transfers >= max_transfers:
#                     continue
                    
#                 # Calculate new fare
#                 new_fare = total_fare + info['min_price']
#                 if new_fare > initial_fare:
#                     continue
                    
#                 # Find next possible stops on this route
#                 current_idx = list(info['stops']).index(current_stop)
#                 next_stops = list(info['stops'])[current_idx + 1:]
                
#                 for next_stop in next_stops:
#                     state = (next_stop, route_id)
#                     if state not in visited:
#                         visited.add(state)
#                         new_transfers = transfers
#                         if path and path[-1][0] != route_id:
#                             new_transfers += 1
                        
#                         new_path = path + [(route_id, next_stop)]
#                         queue.append((next_stop, new_path, new_fare, new_transfers))
                        
#     # Return the path with minimum fare that reaches the destination
#     if valid_paths:
#         valid_paths.sort(key=lambda x: x[1])  # Sort by fare
#         return valid_paths[0][0]  # Return path with minimum fare
#     return []

In [262]:
# Function to prune fare data based on an initial fare limit
def prune_data(merged_fare_df, initial_fare):
    """
    Filter fare data based on an initial fare limit.
    Args:
        merged_fare_df (DataFrame): The merged fare DataFrame.
        initial_fare (float): The maximum fare allowed.
    Returns:
        DataFrame: A filtered DataFrame containing only routes within the fare limit.
    """
    return merged_fare_df[merged_fare_df['cost'] <= initial_fare]

# Pre-computation of Route Summary
def compute_route_summary(pruned_df):
    """
    Generate a summary of routes based on fare information.
    Args:
        pruned_df (DataFrame): The filtered DataFrame containing fare information.
    Returns:
        dict: A summary of routes with the following structure:
        {
            route_id (int): {
                'min_price': float, # The minimum fare for the route
                'stops': set        # A set of stop IDs for that route
            }
        }
    """
    summary = {}
    for _, row in pruned_df.iterrows():
        route_id = row['route_id']
        fare = row['cost']
        
        if route_id not in summary:
            summary[route_id] = {
                'min_price': fare,
                'stops': set(route_to_stops[route_id])
            }
        else:
            summary[route_id]['min_price'] = min(summary[route_id]['min_price'], fare)

    return summary

# PDDL-style planning for route finding
def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Implement PDDL-style planning to find routes with optional transfers.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID for a transfer.
        max_transfers (int): The maximum number of transfers allowed.
    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria, where each tuple contains:
              - route_id (int): The ID of the route.
              - stop_id (int): The ID of the stop.
    """
    paths = []
    
    def explore_path(current_stop, transfers_used, current_path):
        if current_stop == end_stop_id:
            paths.append(current_path)
            return
        if transfers_used > max_transfers:
            return
        
        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for stop in stops:
                    if stop != current_stop and (stop_id_to_include is None or stop_id_to_include in stops):
                        explore_path(stop, transfers_used + 1, current_path + [(route_id, stop)])

    explore_path(start_stop_id, 0, [])
    return paths

# BFS for optimized route planning
def bfs_route_planner_optimized(start_stop_id, end_stop_id, initial_fare, route_summary, max_transfers=3):
    """
    Use Breadth-First Search (BFS) to find the optimal route while considering fare constraints.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        initial_fare (float): The available fare for the trip.
        route_summary (dict): A summary of routes with fare and stop information.
        max_transfers (int): The maximum number of transfers allowed (default is 3).
    Returns:
        list: A list representing the optimal route with stops and routes taken, structured as:
        [
            (route_id (int), stop_id (int)), # Tuple for each stop taken in the route
            ...
        ]
    """
    queue = deque([(start_stop_id, [], 0)])  # (current_stop, path, transfers_used)
    visited = set()
    optimal_route = None

    while queue:
        current_stop, path, transfers_used = queue.popleft()

        if current_stop == end_stop_id:
            if optimal_route is None or len(path) < len(optimal_route):
                optimal_route = path
            continue
        
        if (current_stop, transfers_used) in visited:
            continue
        visited.add((current_stop, transfers_used))

        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for next_stop in stops:
                    if next_stop != current_stop:
                        fare = route_summary[route_id]['min_price']
                        if fare <= initial_fare:
                            queue.append((next_stop, path + [(route_id, next_stop)], transfers_used + 1 if next_stop == stop_id_to_include else transfers_used))

    return optimal_route or []

In [274]:
from collections import deque

# Function to prune fare data based on an initial fare limit
def prune_data(merged_fare_df, initial_fare):
    """
    Filter fare data based on an initial fare limit.
    Args:
        merged_fare_df (DataFrame): The merged fare DataFrame.
        initial_fare (float): The maximum fare allowed.
    Returns:
        DataFrame: A filtered DataFrame containing only routes within the fare limit.
    """
    # Assuming the fare column is named 'fare' instead of 'cost'
    return merged_fare_df[merged_fare_df['fare'] <= initial_fare]

# Pre-computation of Route Summary
def compute_route_summary(pruned_df, route_to_stops):
    """
    Generate a summary of routes based on fare information.
    Args:
        pruned_df (DataFrame): The filtered DataFrame containing fare information.
        route_to_stops (dict): Mapping of route_ids to their stops.
    Returns:
        dict: A summary of routes with the following structure:
        {
            route_id (int): {
                'min_price': float, # The minimum fare for the route
                'stops': set        # A set of stop IDs for that route
            }
        }
    """
    summary = {}
    for _, row in pruned_df.iterrows():
        route_id = row['route_id']
        fare = row['fare']  # Changed from 'cost' to 'fare'
        
        if route_id not in summary:
            summary[route_id] = {
                'min_price': fare,
                'stops': set(route_to_stops[route_id])
            }
        else:
            summary[route_id]['min_price'] = min(summary[route_id]['min_price'], fare)

    return summary

# PDDL-style planning for route finding
def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Implement PDDL-style planning to find routes with optional transfers.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID for a transfer.
        max_transfers (int): The maximum number of transfers allowed.
        route_to_stops (dict): Mapping of route_ids to their stops.
    Returns:
        list: A list of unique paths (list of tuples) that satisfy the criteria.
    """
    paths = []
    
    def explore_path(current_stop, transfers_used, current_path):
        if current_stop == end_stop_id:
            paths.append(current_path)
            return
        if transfers_used > max_transfers:
            return
        
        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for stop in stops:
                    if stop != current_stop and (stop_id_to_include is None or stop_id_to_include in stops):
                        explore_path(stop, transfers_used + 1, current_path + [(route_id, stop)])

    explore_path(start_stop_id, 0, [])
    return paths

# BFS for optimized route planning
def bfs_route_planner_optimized(start_stop_id, end_stop_id, initial_fare, route_summary, route_to_stops, stop_id_to_include=None, max_transfers=3):
    """
    Use Breadth-First Search (BFS) to find the optimal route while considering fare constraints.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        initial_fare (float): The available fare for the trip.
        route_summary (dict): A summary of routes with fare and stop information.
        route_to_stops (dict): Mapping of route_ids to their stops.
        stop_id_to_include (int, optional): Required transfer stop.
        max_transfers (int): The maximum number of transfers allowed (default is 3).
    Returns:
        list: A list representing the optimal route with stops and routes taken.
    """
    queue = deque([(start_stop_id, [], 0)])  # (current_stop, path, transfers_used)
    visited = set()
    optimal_route = None

    while queue:
        current_stop, path, transfers_used = queue.popleft()

        if current_stop == end_stop_id:
            if optimal_route is None or len(path) < len(optimal_route):
                optimal_route = path
            continue
        
        if transfers_used > max_transfers:
            continue
            
        if (current_stop, transfers_used) in visited:
            continue
        visited.add((current_stop, transfers_used))

        for route_id, route_info in route_summary.items():
            if current_stop in route_info['stops']:
                for next_stop in route_info['stops']:
                    if next_stop != current_stop:
                        fare = route_info['min_price']
                        if fare <= initial_fare:
                            new_transfers = transfers_used + 1 if next_stop == stop_id_to_include else transfers_used
                            queue.append((next_stop, path + [(route_id, next_stop)], new_transfers))

    return optimal_route or []

In [285]:
# Function to prune fare data based on an initial fare limit
def prune_data(merged_fare_df, initial_fare):
    """
    Filter fare data based on an initial fare limit.
    Args:
        merged_fare_df (DataFrame): The merged fare DataFrame.
        initial_fare (float): The maximum fare allowed.
    Returns:
        DataFrame: A filtered DataFrame containing only routes within the fare limit.
    """
    return merged_fare_df[merged_fare_df['cost'] <= initial_fare]

# Pre-computation of Route Summary
def compute_route_summary(pruned_df):
    """
    Generate a summary of routes based on fare information.
    Args:
        pruned_df (DataFrame): The filtered DataFrame containing fare information.
    Returns:
        dict: A summary of routes with the following structure:
        {
            route_id (int): {
                'min_price': float, # The minimum fare for the route
                'stops': set        # A set of stop IDs for that route
            }
        }
    """
    summary = {}
    for _, row in pruned_df.iterrows():
        route_id = row['route_id']
        fare = row['cost']
        
        if route_id not in summary:
            summary[route_id] = {
                'min_price': fare,
                'stops': set(route_to_stops[route_id])
            }
        else:
            summary[route_id]['min_price'] = min(summary[route_id]['min_price'], fare)

    return summary

# PDDL-style planning for route finding
def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Implement PDDL-style planning to find routes with optional transfers.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID for a transfer.
        max_transfers (int): The maximum number of transfers allowed.
    Returns:
        list: A list of tuples where each tuple contains:
              (route_id, stop_id_to_include, final_stop)
    """
    paths = []
    
    def explore_path(current_stop, transfers_used, current_path):
        if current_stop == end_stop_id and stop_id_to_include in [stop for _, stop in current_path]:
            # Convert the path format to match expected output
            # We only need the route that includes the required stop and reaches the destination
            for i in range(len(current_path)):
                route_id, stop = current_path[i]
                if stop == stop_id_to_include:
                    paths.append((route_id, stop_id_to_include, end_stop_id))
            return
            
        if transfers_used > max_transfers:
            return
        
        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for stop in stops:
                    if stop != current_stop:
                        new_path = current_path + [(route_id, stop)]
                        explore_path(stop, transfers_used + 1, new_path)

    explore_path(start_stop_id, 0, [])
    
    # Remove duplicates while preserving order
    unique_paths = list(dict.fromkeys(paths))
    return unique_paths

# BFS for optimized route planning
def bfs_route_planner_optimized(start_stop_id, end_stop_id, initial_fare, route_summary, max_transfers=3):
    """
    Use Breadth-First Search (BFS) to find the optimal route while considering fare constraints.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        initial_fare (float): The available fare for the trip.
        route_summary (dict): A summary of routes with fare and stop information.
        max_transfers (int): The maximum number of transfers allowed (default is 3).
    Returns:
        list: A list representing the optimal route with stops and routes taken, structured as:
        [
            (route_id (int), stop_id (int)), # Tuple for each stop taken in the route
            ...
        ]
    """
    queue = deque([(start_stop_id, [], 0)])  # (current_stop, path, transfers_used)
    visited = set()
    optimal_route = None

    while queue:
        current_stop, path, transfers_used = queue.popleft()

        if current_stop == end_stop_id:
            if optimal_route is None or len(path) < len(optimal_route):
                optimal_route = path
            continue
        
        if (current_stop, transfers_used) in visited:
            continue
        visited.add((current_stop, transfers_used))

        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for next_stop in stops:
                    if next_stop != current_stop:
                        fare = route_summary[route_id]['min_price']
                        if fare <= initial_fare:
                            queue.append((next_stop, path + [(route_id, next_stop)], transfers_used + 1 if next_stop == stop_id_to_include else transfers_used))

    return optimal_route or []

In [287]:
test_pddl_planning()


Test pddl_planning (22540, 2573, 4686, 1):  Fail (Expected: [(10153, 4686, 1407)], Got: [(10153, 4686, 2573)])
Test pddl_planning (951, 340, 300, 1):  Fail (Expected: [(294, 300, 712), (10453, 300, 712), (1211, 300, 712), (1158, 300, 712), (37, 300, 712), (1571, 300, 712), (49, 300, 712), (387, 300, 712), (1206, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)], Got: [(1038, 300, 340), (10433, 300, 340), (10453, 300, 340), (1211, 300, 340), (121, 300, 340), (1571, 300, 340), (37, 300, 340), (387, 300, 340), (49, 300, 340)])


In [292]:
# PDDL-style planning for route finding
def pddl_planning(start_stop_id, end_stop_id, stop_id_to_include, max_transfers):
    """
    Implement PDDL-style planning to find routes with optional transfers.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        stop_id_to_include (int): The stop ID for a transfer.
        max_transfers (int): The maximum number of transfers allowed.
    Returns:
        list: A list of tuples where each tuple contains:
              (route_id, stop_id_to_include, final_stop)
    """
    paths = []
    
    def explore_path(current_stop, transfers_used, current_path):
        if current_stop == end_stop_id and stop_id_to_include in [stop for _, stop in current_path]:
            # Convert the path format to match expected output
            # We only need the route that includes the required stop and reaches the destination
            for i in range(len(current_path)):
                route_id, stop = current_path[i]
                if stop == stop_id_to_include:
                    paths.append((route_id, stop_id_to_include, end_stop_id))
            return
            
        if transfers_used > max_transfers:
            return
        
        for route_id, stops in route_to_stops.items():
            if current_stop in stops:
                for stop in stops:
                    if stop != current_stop:
                        new_path = current_path + [(route_id, stop)]
                        explore_path(stop, transfers_used + 1, new_path)

    explore_path(start_stop_id, 0, [])
    
    # Remove duplicates while preserving order
    unique_paths = list(dict.fromkeys(paths))
    return unique_paths

def prune_data(merged_fare_df, initial_fare):
    """
    Filter fare data based on an initial fare limit.
    Args:
        merged_fare_df (DataFrame): The merged fare DataFrame.
        initial_fare (float): The maximum fare allowed.
    Returns:
        DataFrame: A filtered DataFrame containing only routes within the fare limit.
    """
    # First, let's find the fare column - it might be 'price', 'fare_amount', etc.
    fare_column = None
    possible_fare_columns = ['price', 'fare', 'cost', 'fare_amount', 'amount']
    
    for col in possible_fare_columns:
        if col in merged_fare_df.columns:
            fare_column = col
            break
    
    if fare_column is None:
        # If we can't find a fare column, return the original DataFrame
        return merged_fare_df
        
    return merged_fare_df[merged_fare_df[fare_column] <= initial_fare]

def compute_route_summary(pruned_df):
    """
    Generate a summary of routes based on fare information.
    Args:
        pruned_df (DataFrame): The filtered DataFrame containing fare information.
    Returns:
        dict: A summary of routes with fare and stop information
    """
    summary = {}
    # First, find the fare column
    fare_column = None
    possible_fare_columns = ['price', 'fare', 'cost', 'fare_amount', 'amount']
    
    for col in possible_fare_columns:
        if col in pruned_df.columns:
            fare_column = col
            break
    
    if fare_column is None:
        # If we can't find a fare column, use a default value
        default_fare = 1.0
        
    for _, row in pruned_df.iterrows():
        route_id = row['route_id']
        fare = row[fare_column] if fare_column else default_fare
        
        if route_id not in summary:
            summary[route_id] = {
                'min_price': fare,
                'stops': set(route_to_stops[route_id])
            }
        else:
            summary[route_id]['min_price'] = min(summary[route_id]['min_price'], fare)

    return summary

def bfs_route_planner_optimized(start_stop_id, end_stop_id, initial_fare, route_summary, max_transfers=3):
    """
    Use Breadth-First Search (BFS) to find the optimal route while considering fare constraints.
    Args:
        start_stop_id (int): The starting stop ID.
        end_stop_id (int): The ending stop ID.
        initial_fare (float): The available fare for the trip.
        route_summary (dict): A summary of routes with fare and stop information.
        max_transfers (int): The maximum number of transfers allowed (default is 3).
    Returns:
        list: A list representing the optimal route with stops and routes taken.
    """
    queue = deque([(start_stop_id, [], 0, 0)])  # (current_stop, path, transfers_used, total_fare)
    visited = set()
    optimal_route = None

    while queue:
        current_stop, path, transfers_used, total_fare = queue.popleft()

        if current_stop == end_stop_id:
            if optimal_route is None or len(path) < len(optimal_route):
                optimal_route = path
            continue
        
        if transfers_used > max_transfers:
            continue
            
        if (current_stop, transfers_used) in visited:
            continue
        visited.add((current_stop, transfers_used))

        for route_id, info in route_summary.items():
            if current_stop in info['stops']:
                new_fare = total_fare + info['min_price']
                if new_fare <= initial_fare:
                    for next_stop in info['stops']:
                        if next_stop != current_stop:
                            new_transfers = transfers_used + 1 if path and path[-1][0] != route_id else transfers_used
                            if new_transfers <= max_transfers:
                                queue.append((
                                    next_stop, 
                                    path + [(route_id, next_stop)], 
                                    new_transfers,
                                    new_fare
                                ))

    return optimal_route or []

In [293]:
test_bfs_route_planner()

Test bfs_route_planner_optimized (22540, 2573, 10, 3):  Pass
Test bfs_route_planner_optimized (4012, 4013, 10, 3):  Pass


In [294]:
test_pddl_planning()

Test pddl_planning (22540, 2573, 4686, 1):  Fail (Expected: [(10153, 4686, 1407)], Got: [(10153, 4686, 2573)])
Test pddl_planning (951, 340, 300, 1):  Fail (Expected: [(294, 300, 712), (10453, 300, 712), (1211, 300, 712), (1158, 300, 712), (37, 300, 712), (1571, 300, 712), (49, 300, 712), (387, 300, 712), (1206, 300, 712), (1038, 300, 712), (10433, 300, 712), (121, 300, 712)], Got: [(1038, 300, 340), (10433, 300, 340), (10453, 300, 340), (1211, 300, 340), (121, 300, 340), (1571, 300, 340), (37, 300, 340), (387, 300, 340), (49, 300, 340)])


NameError: name 'create_kb' is not defined